Імпортуємо необхідні пакети

In [ ]:
import os
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
plt.style.use('ggplot')  # красиві графіки

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras import optimizers
from keras import initializers
from keras import activations
from keras import callbacks

from sklearn.model_selection import train_test_split

#import sys # якщо в Conda не встановлено keras
#!conda install --yes --prefix {sys.prefix} keras

# Корисні функції

In [ ]:
# Розділяємо дані по класах, що зазначені у третьому стовпці (для візуалізації)
def separate_by_unique_class(data_arr, unique_class, data_dims):
    return data_arr[data_arr[data_dims] == unique_class]

def add_data_to_plot(x, y):
    plt.plot(x, y, 'o')
    
# візуалізація всіх даних з файлу по класах
def add_dots_to_plot(data_f, data_dimentions:int, data_uniq_cls:list):
    
    # тільки якщо ми маємо справу із двовимірними даними
    if data_dimentions == 2:
        
        # проходимо по всіх унікальних класах 
        for d_cl in data_uniq_cls:
            data = separate_by_unique_class(data_f, d_cl, data_dimentions)
            
            add_data_to_plot(data[0], data[1]) 
    else:
        pass
    
# перетворення імовірностей передбачення моделі у номери класів (які модель передбачила)
def transform_predictions(predictions)->list:
    result = []
    prd = predictions.tolist()
    for row in prd:
        result.append(row.index(max(row)))
    return result

# отримуємо ваги напряму із моделі
def _get_weights_list(model)->list:
    weights = []
    for layer in model.layers:
        weights=layer.get_weights()
    return weights

# вирізати колонку під номером "column_num" із масиву "in_list"
def _slice_column(in_list:list, column_num:int)->list:
    lst = []
    for i in in_list:
        lst.append(i[column_num])
    return lst

# перетворємо ваги із формату двох векторів - вектор вагів моделі, вектор вагів bias нейронів у матрицю
def convert_weights_list(weights_raw:list)->list:
    weights_result = []

    weights_result.append(weights_raw[1].tolist()) # ваги нейронів зміщення
    # ваги всіх інших нейронів
    for w in weights_raw[0]:
        weights_result.append(w.tolist())
    return weights_result

# формуємо фінальний масив вагів де рядки відповідають класам а стовпці нейронам (перший - bias)
def form_weights_list(weights_in:list)->list:
    weights_res = []

    for i in range(len(weights_in)):
        weights_res.append(_slice_column(weights_in, i))
    return weights_res

# вирахування ліній для відбраження на графіках
# лише для двовимірних даних
def calc_one_trand_line(x_data:list, weights:list)->list:
    w0 = weights[0]
    w1 = weights[1]
    w2 = weights[2]

    x2 = []
    for x1 in x_data:
        val = (-1)*(w1/w2)*x1 - (w0/w2)
        x2.append(val)
    return x2

# обрахування ліній розділення даних для всіх класів
def calc_trand_lines_list(x_data:list, weights:list)->list:
    x2_arr = []

    for w in weights:
        x2_arr.append(calc_one_trand_line(x_data, w))
    return x2_arr

# кравива візуалізація розділення даних натренованою моделлю
# https://towardsdatascience.com/applied-deep-learning-part-1-artificial-neural-networks-d7834f67a4f6#106c
def plot_multiclass_decision_boundary(model, X, y):
    x_min, x_max = X[:, 0].min() - 0.1, X[:, 0].max() + 0.1
    y_min, y_max = X[:, 1].min() - 0.1, X[:, 1].max() + 0.1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 101), np.linspace(y_min, y_max, 101))
    cmap = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

    Z = model.predict_classes(np.c_[xx.ravel(), yy.ravel()], verbose=0)
    Z = Z.reshape(xx.shape)
    fig = plt.figure(figsize=(8, 8))
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.8)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.RdYlBu)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())

------------------

Місце розташування jupyter notebook

In [ ]:
pwd

In [ ]:
path = "sample1.csv"
data_file = pd.read_csv(path, header=None)
data_file.head()

Запам'ятовуємо назву стовпця, що відповідає класові даних

In [ ]:
data_dimention = len(data_file.columns) - 1  # розмірність вхідних даних, відповідає індексу стовпця клас
data_dimention

Для візуалізації виділяємо унікальні класи даних, для розділення точок на графіку за кольором

In [ ]:
data_unique_classes = data_file[data_dimention].unique()
data_unique_classes

Демонстрація графіку з точками

In [ ]:
plt.figure(figsize=(15,15)) # налаштовуємо розміри відображення даних 
add_dots_to_plot(data_file, data_dimention, data_unique_classes)
plt.show()

# Підготовка даних для передачі в нейромережу

In [ ]:
data_file = data_file.iloc[np.random.permutation(len(data_file))] # перемішування всього масиву значень з csv файлу по рядках
# можна зробити і за допомогою np.random.shuffle(dataset)

# Отримуємо всю таблицю даних з масиву pandas
dataset = data_file.values

# Вирізаємо лише стовпці з векторами
data_array = dataset[:, 0:data_dimention]
# print(data_array)

# вирізаємо стовбець з класами
data_classes = dataset[:, -1]

data_train, data_test, data_classes_train, data_classes_test = train_test_split(
    data_array,
    data_classes,
    test_size=.1
)



---



# Нейронна мережа

Параметри налаштування ініціалізації нейронної мережі

In [ ]:
input_dimention = data_dimention 
output_dimention = len(data_unique_classes) # по скільком класам буде вихід нейромережі

# Найкраще для першого набору даних
epoch_max = 150

batch_size = 128
shuffle_train = True
validation_split = .1
activation = activations.sigmoid

learning_rate = 0.1

kernel_initializer = initializers.TruncatedNormal()#RandomNormal()

patience = 20

Оптимізатор для компіляції моделі

In [ ]:
# https://keras.io/optimizers/

adam = optimizers.Adam(lr=learning_rate)

# стохастичний градієнтний спуск
sgd = optimizers.SGD(lr=learning_rate, momentum=0.9)

rmsprop = optimizers.RMSprop(lr=learning_rate)

###################################################################################################
compile_optimizator = rmsprop

Створення моделі

In [ ]:

model = Sequential()

model.add(Dense(
    output_dim=output_dimention,
    input_dim=input_dimention,
    activation= activation,
    kernel_initializer = kernel_initializer
))

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=compile_optimizator,
    metrics=[
        'accuracy'
    ]
)

#model.summary()  # демонстрація структури моделі

Колбеки для більшого контролю за процесом навчання

In [ ]:
# https://datascience.stackexchange.com/questions/37186/early-stopping-on-validation-loss-or-on-accuracy

# якщо протягом певної кількості епох метрика не змінюватиметься,
# тобто модель почне перенавчання (overfitting), процес тренування зупиниться
early_stop = callbacks.EarlyStopping(
    monitor = 'val_loss',
    patience = patience
)

# Фінальний список колбеків для передачі у функцію тренування
callback_list = [
    callbacks.TerminateOnNaN(),
    early_stop
]

# Навчання нейронної мережі

In [ ]:
history = model.fit(
    x = data_train,
    y = data_classes_train,
    epochs = epoch_max,
    batch_size = batch_size,
    shuffle = shuffle_train,
    validation_data = (data_test, data_classes_test),
    verbose = 1,
    callbacks = callback_list
)    

-----------------------------

# Перевірка точності моделі на тестовому наборі даних

In [ ]:
results = model.evaluate(data_test, data_classes_test)

print(model.metrics_names)
print(results)

---

# Візуалізація процесу навчання моделі

Виведемо список доступних метрик для відстеження

In [ ]:
# продемонструємо доступні метрики для візуалізації якості навчання
# список метрик змінюватиметься в залежності від вибраних метрик на етапі компіляції моделі
for history_metric in history.history:
    print(history_metric)

Візуалізація

In [ ]:
# Формування списків метрик для візуалізації якості навчання
loss = history.history['loss']  # наскільки точно модель 'впізнає' дані із тренувального набору
val_loss = history.history['val_loss']  # точність розпізнання даних із тестового набору

accuracy = history.history['acc']  # кількість невірно розпізнаних даних на тренувальному наборі
val_accuracy = history.history['val_acc']  # кількість невірно розпізнаних даних на тестовому наборі

epochs_passed = range(1, len(loss) + 1)  # кількість епох навчання моделі

plt.figure(figsize=(10,10))

plt.plot(epochs_passed, loss, label='Training loss')
plt.plot(epochs_passed, val_loss, label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.figure(figsize=(10,10))

plt.plot(epochs_passed, accuracy, label='Training accuracy')
plt.plot(epochs_passed, val_accuracy, label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()



---



# Ваги моделі

Отримуємо ваги з натренованої моделі

In [ ]:
raw_weights = _get_weights_list(model)
weights = convert_weights_list(raw_weights)

for i in weights:
    print(i)
print("---------------------------------")

# конвертація вагів у зручний для роботи формат
weights = form_weights_list(weights)

for i in weights:
    print(i)


# Візуалізація результату навчання

In [ ]:
x1 = [-20,-15, -10, -5, 0, 5, 10, 15, 20]
x2_list = list(calc_trand_lines_list(x1, weights))

# for x_line in x2_list:
#     for i in range(len(x1)):
#         print(f"x1 = {x1[i]}   x2 = {x_line[i]}")
#     print("<----------->")

Візуалізуємо розділення класів даних

In [ ]:
plt.figure(figsize=(15,15)) # налаштовуємо розміри відображення даних 
add_dots_to_plot(data_file, data_dimention, data_unique_classes)

axes = plt.gca()
axes.set_xlim([-10,20])
axes.set_ylim([-25,25])

for x2_line in x2_list:
    plt.plot(x1, x2_line)
    
plt.show()

Розподілення рішень нейромережі щодо даних

In [ ]:
plt.figure(figsize=(15,15))

plot_multiclass_decision_boundary(model, data_array, data_classes)

Передбачення класів на основі даних натренованою нейромережею

In [ ]:
predictions = model.predict(data_test)
# print(predictions)

In [ ]:
# перетворення передбаченого вектору імовірностей у клас даних
pred_transformed = transform_predictions(predictions)

for i in range(len(data_test)):
    print("----")
    print("data {0}  real class {1}  predicted class {2}".format(data_test[i], data_classes_test[i], pred_transformed[i]))

In [ ]:
## https://www.machinecurve.com/index.php/2019/07/24/why-you-cant-truly-create-rosenblatts-perceptron-with-keras/
## https://www.kaggle.com/arihant0497/try-shallow-before-going-deep
## https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/

## https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

# Найкраще для першого набору даних
epoch_max = 200

batch_size = 128

shuffle_train = True

validation_split = .1

activation = activations.sigmoid

learning_rate = 0.01

kernel_initializer = initializers.TruncatedNormal()

patience = 10

Ітерація 2##

epoch_max = 500

batch_size = 128

shuffle_train = True

validation_split = .1

activation = activations.sigmoid

learning_rate = 0.1

kernel_initializer = initializers.TruncatedNormal()

patience = 20
